In [ ]:
%pip install langchain --quiet
%pip install chromadb --quiet
%pip install tiktoken --quiet
%pip install transformers --quiet
%pip install xformer --quiet
%pip install sentence-transformers --quiet
%pip install openai --quiet

### Paraméterek

In [ ]:
# használt erőforrások mappája
res_folder = "res/in_use/"
    
# splitting paraméterek
chunk_size = 500
chunk_overlap = 50

# vector store paraméterek
persist_directory = "res/chroma/"
search_type = "mmr"
search_k = 5
search_fetch_k = 8
lambda_mult = 0.6

# memória
memory_k = 3

# ChatGPT paraméterek
temperature = 0.4
max_tokens = 500
model_id = "gpt-3.5-turbo"

## Erőforrásfájlok betöltése

Ha már egyszer megtettük és van mentett vektor adatbázis, akkor nem kell újra futtatni.
### CSV fájlok (nagyrészt kérdések) betöltése

In [ ]:
from langchain.document_loaders import CSVLoader, DirectoryLoader

directory_loader = DirectoryLoader(res_folder, glob="*.csv", use_multithreading=True, loader_cls=CSVLoader, loader_kwargs={"encoding": "utf-8"})
csv_data = directory_loader.load()
print(len(csv_data))
#csv_data


### Szöveges fájlok betöltése, majd feldarabolása

In [ ]:
from langchain.document_loaders import DirectoryLoader, TextLoader

directory_loader = DirectoryLoader(res_folder, glob="*.txt", use_multithreading=True, loader_cls=TextLoader, loader_kwargs={"encoding": "utf-8"})
text_data = directory_loader.load()
len(text_data)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    length_function=len,
    is_separator_regex=True,
    separators=["\n\s*\n", "\n\s*", "\n"]
)

split_text_data = text_splitter.split_documents(text_data)
print(len(split_text_data))
#split_text_data

### VectorStore inicializálása Chroma-val

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores.chroma import Chroma

#embedding = HuggingFaceEmbeddings(model_name= "sentence-transformers/all-mpnet-base-v2")
embedding = OpenAIEmbeddings()

In [ ]:
# Elég egyszer futtatni, ha nem változtatunk az adatokon, mert lementi a vektoradatbázist.
combined_data = []
combined_data.extend(split_text_data)
combined_data.extend(csv_data)

vectordb = Chroma.from_documents(
    documents=combined_data,
    embedding=embedding,
    persist_directory=persist_directory
)

vectordb.persist()

Ha le van már mentve, lehet ezt használni az előző cella  helyett:

In [ ]:
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

In [ ]:
# MMR teszt
question = "Ki a tárgyfelelős?"
search_result = vectordb.max_marginal_relevance_search(question,k = search_k, fetch_k = search_fetch_k, lambda_mult = lambda_mult)
print(search_result)
print(vectordb.similarity_search_with_relevance_scores(question))

### Memória config a chat historyhoz

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k = memory_k, memory_key="chat_history", return_messages=True)

## Chatbot Puli GPTrioval

### ConversationalRetrievalChain használatával

In [ ]:
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import HuggingFacePipeline
from transformers import pipeline, GPTNeoXForCausalLM, AutoTokenizer
from langchain.chains import StuffDocumentsChain, LLMChain, ConversationalRetrievalChain
from langchain.prompts import PromptTemplate

model_name = "NYTK/PULI-GPTrio"
model = GPTNeoXForCausalLM.from_pretrained(model_name, temperature=0.4, max_length=50, top_k=20, top_p=1)
tokenizer = AutoTokenizer.from_pretrained(model_name)
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer)

In [ ]:
llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
template = """A chat előzményekből és a következő inputból alakíts ki egy önmagában is helytálló kérdést, ha az input értelmezéséhez fontos kontextust tartalmaz!
Ha az input nem kérdés vagy nem kapcsolódik már az előzményekhez, akkor csak add vissza az inputot!

Chat előzmények:
{chat_history}

Következő input: {question}

Átalakított kérdés:"""
question_generator_prompt = PromptTemplate.from_template(template)
question_generator_chain = LLMChain(
    llm=llm,
    prompt=question_generator_prompt,
    verbose=True
)

In [ ]:
qa_template = """A BME VIK szakmai gyakorlat kérdéseire válaszoló chatbot vagy. A kérdésekre magyarul válaszolj!
Használd az alábbi dokumentumrészleteket forrásként a felhasználó kérdésének megválaszolásához!
Ha azokból nem tudsz megadni releváns választ, akkor válaszold azt, hogy "Sajnos erre nem tudok válaszolni, kérdezz mást a BME VIK szakmai gyakorlattal kapcsolatban"

DOKUMENTUMRÉSZLETEK:
{context}

ÚJ INPUT: {question}

Válaszolj a kérdésre!"""

qa_prompt = PromptTemplate.from_template(qa_template)
llm_chain = LLMChain(
    llm=llm,
    prompt=qa_prompt,
    verbose=True
)
combine_docs_chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_variable_name="context",
    verbose=True
)

In [ ]:
memory.clear()
chain = ConversationalRetrievalChain(
    combine_docs_chain=combine_docs_chain,
    retriever = vectordb.as_retriever(
        searh_type = search_type,
        search_kwargs = {
            "k": search_k,
            "fetch_k": search_fetch_k,
            "lambda_mult": lambda_mult
        }
    ),
    question_generator=question_generator_chain,
    memory=memory,
    #return_generated_question=True,
    rephrase_question=True,
    verbose=True
)

## Tesztelés

#### Teszt kérdések betöltése

In [ ]:
import pandas as pd

test_questions = pd.read_csv("testing/test_questions.csv")
print(len(test_questions))
#test_questions

In [ ]:
%pip install pandas --quiet
%pip install matplotlib --quiet

In [ ]:
import pandas as pd
df = pd.DataFrame(columns=['Question', 'Document', 'Score'])
chain.verbose = True

for question in test_questions:
    search_result = vectordb.similarity_search_with_relevance_scores(question, k = search_k)
    for res in search_result:
        df.loc[len(df)] = [question, res[0], res[1]]

df

In [ ]:
# elmentés
df.to_csv(f"testing/results_k{search_k}_size{chunk_size}.csv", index=False)

#### Gráf

In [ ]:
import matplotlib.pyplot as plt

grouped = df.groupby('Question')

fig, ax = plt.subplots()

# Iterate over each group
for i, (name, group) in enumerate(grouped, start=1):
    group = group.reset_index(drop=True)
    ax.plot(group.index+1, group['Score'], label=f"Group {i}")

ax.legend(fontsize=8, loc='upper right')
plt.xlabel('Index')
plt.xlim(0.75, search_k+0.25)
plt.xticks(range(1, search_k+1))

plt.ylabel('Score')
plt.ylim(min(df['Score']) - 0.005, max(df['Score']) + 0.005)

plt.title('Score for Each Question')
plt.show()

In [ ]:
memory.buffer

In [ ]:
# Ha törölni szeretnénk a memóriát
memory.clear()

## Gradio UI a chatbothoz

In [ ]:
%pip install gradio --quiet

In [ ]:
def qa(message, history) -> str:
    prompt = f'{message}\nHa a kontextusból nem tudsz értelmes választ adni, akkor írd, hogy "Nem tudok erre a kérdésre válaszolni, tegyél fel valamit a VIK szakmai gyakorlatával kapcsolatban."'
    return chain({"question": prompt, "chat_history": memory.buffer})["answer"]

In [ ]:
import gradio as gr

chain.verbose = True
chat_ui = gr.ChatInterface(qa, title = "Lacibot", description="Kérdezz a VIK-es szakmai gyakorlatról!", undo_btn=None)
chat_ui.launch(debug=True)

In [ ]:
message= question
prompt = f'{message}\nHa a kontextusból nem tudsz értelmes választ adni, akkor írd, hogy "Nem tudok erre a kérdésre válaszolni, tegyél fel valamit a VIK szakmai gyakorlatával kapcsolatban."'
chain({"question": prompt, "chat_history": memory.buffer})